In [16]:
import pandas as pd
from scipy.stats import skew, kurtosis
from scipy.signal import argrelextrema
from scipy.stats import shapiro
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
from scipy.stats import spearmanr
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import folium 
from folium.plugins import StripePattern
import branca.colormap
from collections import defaultdict
from folium.plugins import HeatMap

In [31]:
df= pd.read_csv("C:/Users/asus/Documents/Python/Capstone/SaaS-Sales.csv")

In [18]:
print(df.columns)

Index(['Row ID', 'Order ID', 'Order Date', 'Date Key', 'Contact Name',
       'Country', 'City', 'Region', 'Subregion', 'Customer', 'Customer ID',
       'Industry', 'Segment', 'Product', 'License', 'Sales', 'Quantity',
       'Discount', 'Profit'],
      dtype='object')


In [19]:
print(df.shape) 

(9994, 19)


In [20]:
print(df.duplicated().sum())

0


In [21]:
print(df.isnull().sum())

Row ID          0
Order ID        0
Order Date      0
Date Key        0
Contact Name    0
Country         0
City            0
Region          0
Subregion       0
Customer        0
Customer ID     0
Industry        0
Segment         0
Product         0
License         0
Sales           0
Quantity        0
Discount        0
Profit          0
dtype: int64


In [22]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Row ID        9994 non-null   int64  
 1   Order ID      9994 non-null   object 
 2   Order Date    9994 non-null   object 
 3   Date Key      9994 non-null   int64  
 4   Contact Name  9994 non-null   object 
 5   Country       9994 non-null   object 
 6   City          9994 non-null   object 
 7   Region        9994 non-null   object 
 8   Subregion     9994 non-null   object 
 9   Customer      9994 non-null   object 
 10  Customer ID   9994 non-null   int64  
 11  Industry      9994 non-null   object 
 12  Segment       9994 non-null   object 
 13  Product       9994 non-null   object 
 14  License       9994 non-null   object 
 15  Sales         9994 non-null   float64
 16  Quantity      9994 non-null   int64  
 17  Discount      9994 non-null   float64
 18  Profit        9994 non-null 

In [23]:
df.head()

,Row ID,Order ID,Order Date,Date Key,Contact Name,Country,City,Region,Subregion,Customer,Customer ID,Industry,Segment,Product,License,Sales,Quantity,Discount,Profit
0,1,EMEA-2022-152156,11/9/2022,20221109,Nathan Bell,Ireland,Dublin,EMEA,UKIR,Chevron,1017,Energy,SMB,Marketing Suite,16GRM07R1K,261.9600,2,0.00,41.9136
1,2,EMEA-2022-152156,11/9/2022,20221109,Nathan Bell,Ireland,Dublin,EMEA,UKIR,Chevron,1017,Energy,SMB,FinanceHub,QLIW57KZUV,731.9400,3,0.00,219.5820
2,3,AMER-2022-138688,6/13/2022,20220613,Deirdre Bailey,United States,New York City,AMER,NAMER,Phillips 66,1056,Energy,Strategic,FinanceHub,JI6BVL70HQ,14.6200,2,0.00,6.8714
3,4,EMEA-2021-108966,10/11/2021,20211011,Zoe Hodges,Germany,Stuttgart,EMEA,EU-WEST,Royal Dutch Shell,1031,Energy,SMB,ContactMatcher,DE9GJKGD44,957.5775,5,0.45,-383.0310
4,5,EMEA-2021-108966,10/11/2021,20211011,Zoe Hodges,Germany,Stuttgart,EMEA,EU-WEST,Royal Dutch Shell,1031,Energy,SMB,Marketing Suite - Gold,OIF7NY23WD,22.3680,2,0.20,2.5164


In [32]:
import numpy as np
import pandas as pd

# 1. Safely calculate and format 'Profit Margin' and 'Discount' as percentages
df['Profit Margin'] = np.where(df['Sales'] != 0, df['Profit'] / df['Sales'], 0)
df['Profit Margin'] = (df['Profit Margin'] * 100).round(2)
df['Discount'] = (df['Discount'] * 100).round(2)

# 2. Convert 'Order Date' to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])

# 3. Sort by most recent orders
df = df.sort_values(by='Order Date', ascending=False).reset_index(drop=True)

# 4. Update 'Industry' for specific customers
industry_map = {
    "Glencore": "Mining",
    "News Corp.": "Media"
}
df['Industry'] = df['Customer'].map(industry_map).fillna(df['Industry'])

# 5. Drop unnecessary columns (safe drop if columns exist)
cols_to_drop = ['Row ID', 'Date Key', 'Contact Name', 'License']
clean_df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

# 6. Optional: Check for duplicates (excluding 'Order ID')
duplicates = clean_df[clean_df.duplicated(subset=[col for col in clean_df.columns if col != 'Order ID'])]

# 7. Check a specific Order ID (example)
order_check = clean_df[clean_df['Order ID'] == 'AMER-2020-150119']

# 8. Drop all exact duplicates
clean_df = clean_df.drop_duplicates()

# You can print or return `duplicates` and `order_check` if needed for inspection:
# print(duplicates)
# print(order_check)
clean_df

,Order ID,Order Date,Country,City,Region,Subregion,Customer,Customer ID,Industry,Segment,Product,Sales,Quantity,Discount,Profit,Profit Margin
0,EMEA-2023-143259,2023-12-31,United Kingdom,London,EMEA,UKIR,Morgan Stanley,1068,Finance,SMB,Marketing Suite,323.136,4,20.0,12.1176,3.75
1,EMEA-2023-143259,2023-12-31,United Kingdom,London,EMEA,UKIR,Morgan Stanley,1068,Finance,SMB,Site Analytics,90.930,7,0.0,2.7279,3.00
2,AMER-2023-115427,2023-12-31,United States,Columbia,AMER,NAMER,Aviva,1006,Finance,Strategic,ContactMatcher,13.904,2,20.0,4.5188,32.50
3,AMER-2023-115427,2023-12-31,United States,Columbia,AMER,NAMER,Aviva,1006,Finance,Strategic,ContactMatcher,20.720,2,20.0,6.4750,31.25
4,APJ-2023-126221,2023-12-31,India,Mumbai,APJ,IND,Merck,1064,Healthcare,Enterprise,OneView,209.300,2,0.0,56.5110,27.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,EMEA-2020-141817,2020-01-06,France,Paris,EMEA,EU-WEST,Mitsubishi,1050,Manufacturing,SMB,ChatBot Plugin,19.536,3,20.0,4.8840,25.00
9990,APJ-2020-112326,2020-01-05,Australia,Wollongong,APJ,ANZ,CVS Health,1026,Retail,Enterprise,ContactMatcher,3.540,2,80.0,-5.4870,-155.00
9991,APJ-2020-112326,2020-01-05,Australia,Wollongong,APJ,ANZ,CVS Health,1026,Retail,Enterprise,FinanceHub,11.784,3,20.0,4.2717,36.25
9992,APJ-2020-112326,2020-01-05,Australia,Wollongong,APJ,ANZ,CVS Health,1026,Retail,Enterprise,Marketing Suite - Gold,272.736,3,20.0,-64.7748,-23.75


In [33]:
print(clean_df.info()) 
print(clean_df.duplicated().sum()) 


<class 'pandas.core.frame.DataFrame'>
Index: 9993 entries, 0 to 9993
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       9993 non-null   object        
 1   Order Date     9993 non-null   datetime64[ns]
 2   Country        9993 non-null   object        
 3   City           9993 non-null   object        
 4   Region         9993 non-null   object        
 5   Subregion      9993 non-null   object        
 6   Customer       9993 non-null   object        
 7   Customer ID    9993 non-null   int64         
 8   Industry       9993 non-null   object        
 9   Segment        9993 non-null   object        
 10  Product        9993 non-null   object        
 11  Sales          9993 non-null   float64       
 12  Quantity       9993 non-null   int64         
 13  Discount       9993 non-null   float64       
 14  Profit         9993 non-null   float64       
 15  Profit Margin  9993 non-nu

In [37]:
clean_df[clean_df['Order ID'] == 'AMER-2020-150119']

,Order ID,Order Date,Country,City,Region,Subregion,Customer,Customer ID,Industry,Segment,Product,Sales,Quantity,Discount,Profit,Profit Margin
9605,AMER-2020-150119,2020-04-23,Mexico,Mexico City,AMER,LATAM,Itochu,1032,Finance,Enterprise,FinanceHub,281.372,2,30.0,-12.0588,-4.29
9608,AMER-2020-150119,2020-04-23,Mexico,Mexico City,AMER,LATAM,Itochu,1032,Finance,Enterprise,ContactMatcher,7.488,8,70.0,-5.2416,-70.00
9609,AMER-2020-150119,2020-04-23,Mexico,Mexico City,AMER,LATAM,Itochu,1032,Finance,Enterprise,SaaS Connector Pack,22.336,4,20.0,7.8176,35.00


In [38]:
print(clean_df.isna().sum())

Order ID         0
Order Date       0
Country          0
City             0
Region           0
Subregion        0
Customer         0
Customer ID      0
Industry         0
Segment          0
Product          0
Sales            0
Quantity         0
Discount         0
Profit           0
Profit Margin    0
dtype: int64


In [39]:
clean_df.to_csv('C:/Users/asus/Documents/Python/Capstone/clean_saas.csv', index=False)